In [1]:
using Interact
using CoordinateTransformations
using LCMGL
using ForwardDiff
using RigidBodyDynamics
using DataFrames

 in depwarn(::

String, ::Symbol) at ./deprecated.jl:64
 in readall(::String, ::Vararg{String,N}) at ./deprecated.jl:30
 in include_from_node1(::String) at ./loading.jl:488
 in __init__() at /home/rdeits/locomotion/explorations/point-cloud-signed-distance/packages/v0.5/Interact/src/Interact.jl:81
 in _include_from_serialized(::String) at ./loading.jl:150
 in _require_from_serialized(::Int64, ::Symbol, ::String, ::Bool) at ./loading.jl:187
 in _require_search_from_serialized(::Int64, ::Symbol, ::String, ::Bool) at ./loading.jl:217
 in require(::Symbol) at ./loading.jl:371
 in include_string(::String, ::String) at ./loading.jl:441
 in execute_request(::ZMQ.Socket, ::IJulia.Msg) at /home/rdeits/locomotion/explorations/point-cloud-signed-distance/packages/v0.5/IJulia/src/execute_request.jl:169
 in eventloop(::ZMQ.Socket) at /home/rdeits/locomotion/explorations/point-cloud-signed-distance/packages/v0.5/IJulia/src/eventloop.jl:8
 in (::IJulia.##9#15)() at ./task.jl:360
while loading /home/rdeits/locomotion/

In [2]:
import Flash
reload("Flash")

In [3]:
model = Flash.Models.beanbag()

Flash.Manipulator{Float64}(Vertex: world (root)
  Vertex: body1, Edge: joint1,DataStructures.OrderedDict(RigidBody: "body1"=>Flash.BodyGeometry{Float64}(RigidBodyDynamics.Point3D{Float64}[Point3D in "body1": [-1.0,0.0,0.0],Point3D in "body1": [1.0,0.0,0.0],Point3D in "body1": [0.0,-1.0,0.0],Point3D in "body1": [0.0,1.0,0.0],Point3D in "body1": [0.0,0.0,-1.0],Point3D in "body1": [0.0,0.0,1.0]],RigidBodyDynamics.Point3D{Float64}[Point3D in "body1": [0.0,0.0,0.0]],Flash.DeformableGeometry())),Array{RigidBodyDynamics.RigidBody{Float64},1}[RigidBodyDynamics.RigidBody{Float64}[RigidBody: "body1"]])

In [4]:
state = Flash.ManipulatorState(model)
Flash.draw(state)

DrakeVisualizer.Visualizer(DrakeVisualizer.Robot(DrakeVisualizer.Link[DrakeVisualizer.Link(DrakeVisualizer.GeometryData[DrakeVisualizer.GeometryData{CoordinateTransformations.IdentityTransformation,GeometryTypes.HomogenousMesh{FixedSizeArrays.Point{3,Float64},GeometryTypes.Face{3,Int64,0},Void,Void,Void,Void,Void}}(HomogenousMesh(
    vertices: 5966xFixedSizeArrays.Point{3,Float64},     faces: 11928xGeometryTypes.Face{3,Int64,0}, )
,CoordinateTransformations.IdentityTransformation(),RGBA{Float64}(1.0,0.0,0.0,0.5))],"skin_1")]),1,PyLCM.LCM(PyObject <LCM object at 0x7ff7441ba2d0>))

In [5]:
geometry = collect(values(model.geometries))[1]
@manipulate for x = linspace(-1, 1, 51)
    state.deformations[geometry][1][:] = [x, 0, 0]
    Flash.draw(state)
end

Interact.Slider{Float64}(Signal{Float64}(0.0, nactions=0),"x",0.0,linspace(-1.0,1.0,51),true)

DrakeVisualizer.Visualizer(DrakeVisualizer.Robot(DrakeVisualizer.Link[DrakeVisualizer.Link(DrakeVisualizer.GeometryData[DrakeVisualizer.GeometryData{CoordinateTransformations.IdentityTransformation,GeometryTypes.HomogenousMesh{FixedSizeArrays.Point{3,Float64},GeometryTypes.Face{3,Int64,0},Void,Void,Void,Void,Void}}(HomogenousMesh(
    vertices: 5966xFixedSizeArrays.Point{3,Float64},     faces: 11928xGeometryTypes.Face{3,Int64,0}, )
,CoordinateTransformations.IdentityTransformation(),RGBA{Float64}(1.0,0.0,0.0,0.5))],"skin_1")]),1,PyLCM.LCM(PyObject <LCM object at 0x7ff7441ba300>))

In [6]:
# Construct the sensor and draw its view rays for debugging
sensor = Flash.DepthSensors.Kinect(80,80);
camera_origin = [0., 0, 8]
camera_tform = compose(Translation(camera_origin), LinearMap(AngleAxis(pi, 1, 0, 0)))
Flash.DepthSensors.draw_rays(sensor, camera_tform)

In [7]:
function raycast(state::Flash.ManipulatorState)
    Flash.draw(state)
    surfaces = Flash.skin(state)
    warn("only using the first surface")
    skin = surfaces[1]
    
    points = Flash.DepthSensors.raycast_points(skin, sensor, camera_tform)
    LCMGLClient("raycast") do lcmgl
        LCMGL.color(lcmgl, 0, 1, 0)
        point_size(lcmgl, 5)
        begin_mode(lcmgl, LCMGL.POINTS)
        for point in points
            vertex(lcmgl, convert(Vector, point)...)
        end
        end_mode(lcmgl)
        switch_buffer(lcmgl)
    end
    points
end

raycast (generic function with 1 method)

In [8]:
function flatten(state::Flash.ManipulatorState)
    vcat(state.mechanism_state.q, state.deformation_data)
end

function unflatten!{T}(state::Flash.ManipulatorState, data::AbstractVector{T})
    nq = num_positions(state.mechanism_state)
    set_configuration!(state.mechanism_state, data[1:nq])
    state.deformation_data[:] = data[(nq + 1):end]
end

unflatten! (generic function with 1 method)

In [9]:
true_state = Flash.ManipulatorState(model)
true_state.mechanism_state.q[5:7] = 2 * rand(3)
true_state.deformation_data[:] = 0.5 .* (rand(Float64, length(true_state.deformation_data)) .- 0.5)
Flash.draw(true_state)

DrakeVisualizer.Visualizer(DrakeVisualizer.Robot(DrakeVisualizer.Link[DrakeVisualizer.Link(DrakeVisualizer.GeometryData[DrakeVisualizer.GeometryData{CoordinateTransformations.IdentityTransformation,GeometryTypes.HomogenousMesh{FixedSizeArrays.Point{3,Float64},GeometryTypes.Face{3,Int64,0},Void,Void,Void,Void,Void}}(HomogenousMesh(
    vertices: 7048xFixedSizeArrays.Point{3,Float64},     faces: 14092xGeometryTypes.Face{3,Int64,0}, )
,CoordinateTransformations.IdentityTransformation(),RGBA{Float64}(1.0,0.0,0.0,0.5))],"skin_1")]),1,PyLCM.LCM(PyObject <LCM object at 0x7ff7441ba450>))

In [10]:
x_true = flatten(true_state)

sensed_points = raycast(true_state)

cost = x -> begin
    state = Flash.ManipulatorState(model, eltype(x), eltype(x))
    unflatten!(state, x)
    Flash.draw(state, false)
    skin = Flash.skin(state)[1]
    raycast_cost = sum(point -> skin(point)^2, sensed_points)
    deformation_cost = zero(eltype(x))
    for ld in values(state.deformations)
        for d in ld
            deformation_cost += sum(d .^2)
        end
    end
    return raycast_cost + 10*deformation_cost
end

# Use autodiff to prepare a function that will evaluate the gradient
# of that cost:
cost_and_gradient = x -> begin
    out = GradientResult(x)
    ForwardDiff.gradient!(out, cost, x)
    ForwardDiff.value(out), ForwardDiff.gradient(out)
end

(::#22) (generic function with 1 method)

In [11]:
trials = []

function test_gradient_descent()
    for i = 1:1
        xs = []

        x_estimated = flatten(Flash.ManipulatorState(model))

        for i = 1:500
            push!(xs, x_estimated)

            # Evaluate the gradient of the point values w.r.t. the
            # joint angles:
            ci, gi = cost_and_gradient(x_estimated)

            if any(isnan, gi)
                @show x_estimated
                error("NaN in gradient")
            end

            # Descend the gradient
            x_estimated -= 0.002 * gi
            
            # TODO: generalize this
            x_estimated[1:4] = x_estimated[1:4] / norm(x_estimated[1:4])
        end
        push!(trials, DataFrame(x=xs))
    end
end

test_gradient_descent (generic function with 1 method)

In [12]:
test_gradient_descent()
state = Flash.ManipulatorState(model)
unflatten!(state, trials[1][:x][end])
Flash.draw(state)

DrakeVisualizer.Visualizer(DrakeVisualizer.Robot(DrakeVisualizer.Link[DrakeVisualizer.Link(DrakeVisualizer.GeometryData[DrakeVisualizer.GeometryData{CoordinateTransformations.IdentityTransformation,GeometryTypes.HomogenousMesh{FixedSizeArrays.Point{3,Float64},GeometryTypes.Face{3,Int64,0},Void,Void,Void,Void,Void}}(HomogenousMesh(
    vertices: 142746xFixedSizeArrays.Point{3,Float64},     faces: 285488xGeometryTypes.Face{3,Int64,0}, )
,CoordinateTransformations.IdentityTransformation(),RGBA{Float64}(1.0,0.0,0.0,0.5))],"skin_1")]),1,PyLCM.LCM(PyObject <LCM object at 0x7ff72a3d0750>))

In [13]:
using Ipopt
using SimpleGradientDescent

In [14]:
x0 = flatten(Flash.ManipulatorState(model))
num_vars = length(x0)
solver = IpoptSolver(print_level=1)
m = getmodel(GradientDescent(cost), num_vars, solver)
setwarmstart!(m, x0);

In [15]:
optimize!(m)
x = getsolution(m)
state = Flash.ManipulatorState(model)
unflatten!(state, x)
Flash.draw(state)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************



DrakeVisualizer.Visualizer(DrakeVisualizer.Robot(DrakeVisualizer.Link[DrakeVisualizer.Link(DrakeVisualizer.GeometryData[DrakeVisualizer.GeometryData{CoordinateTransformations.IdentityTransformation,GeometryTypes.HomogenousMesh{FixedSizeArrays.Point{3,Float64},GeometryTypes.Face{3,Int64,0},Void,Void,Void,Void,Void}}(HomogenousMesh(
    vertices: 6548xFixedSizeArrays.Point{3,Float64},     faces: 13092xGeometryTypes.Face{3,Int64,0}, )
,CoordinateTransformations.IdentityTransformation(),RGBA{Float64}(1.0,0.0,0.0,0.5))],"skin_1")]),1,PyLCM.LCM(PyObject <LCM object at 0x7ff72a3637b0>))